## Data Processing

In [1]:
import numpy as np
import pandas as pd
#read weather data
temperature = pd.read_csv('temperature.csv',low_memory=False,index_col = 'datetime')
pressure = pd.read_csv('pressure.csv',low_memory = False,index_col = 'datetime')
humidity = pd.read_csv('humidity.csv',low_memory = False,index_col = 'datetime')
weather_description = pd.read_csv('weather_description.csv',low_memory = False,index_col = 'datetime')
wind_direction = pd.read_csv('wind_direction.csv',low_memory = False,index_col = 'datetime')
wind_speed = pd.read_csv('wind_speed.csv',low_memory = False,index_col = 'datetime')
#read bike flow data
bike_flow = pd.read_csv('bike_flow.csv',low_memory = False,index_col = 'Date')


### Chosing 5 years Seattle weather data

In [2]:
#pick 5 year data
temperature = temperature.loc['2012-11-30 00:00:00':'2017-11-30 00:00:00']
pressure = pressure.loc['2012-11-30 00:00:00':'2017-11-30 00:00:00']
humidity = humidity.loc['2012-11-30 00:00:00':'2017-11-30 00:00:00']
weather_description = weather_description.loc['2012-11-30 00:00:00':'2017-11-30 00:00:00']
wind_direction = wind_direction.loc['2012-11-30 00:00:00':'2017-11-30 00:00:00']
wind_speed = wind_speed.loc['2012-11-30 00:00:00':'2017-11-30 00:00:00']
#pick Seattle
temperature = temperature.filter(items=['datetime','Seattle'])
pressure = pressure.filter(items=['datetime','Seattle'])
humidity = humidity.filter(items=['datetime','Seattle'])
weather_description = weather_description.filter(items=['datetime','Seattle'])
wind_direction = wind_direction.filter(items=['datetime','Seattle'])
wind_speed = wind_speed.filter(items=['datetime','Seattle'])
#Combine them all
weather = (temperature.merge(humidity,on='datetime').merge(pressure,on='datetime').
           merge(weather_description,on='datetime').merge(wind_direction,on='datetime').
          merge(wind_speed,on='datetime')).reset_index()
weather.columns = ['datetime','temperature','humidity','pressure','description','wind_direction','wind_speed']
weather['temperature'] = weather['temperature'] - 273.15
weather.head()

,datetime,temperature,humidity,pressure,description,wind_direction,wind_speed
0,2012-11-30 00:00:00,7.42,87.0,1003.0,light rain,0.0,0.0
1,2012-11-30 01:00:00,7.78,87.0,1003.0,light rain,0.0,1.0
2,2012-11-30 02:00:00,8.13,87.0,1003.0,light rain,160.0,2.0
3,2012-11-30 03:00:00,8.24,64.0,1025.0,sky is clear,270.0,10.0
4,2012-11-30 04:00:00,8.66,68.0,1019.0,overcast clouds,309.0,5.0


### Select the corresponding bike flow data and combine them

In [3]:
bike_flow = bike_flow.loc['11/30/2012 12:00:00 AM':'11/30/2017 12:00:00 AM']
bike_flow = bike_flow.reset_index()

In [4]:
df = pd.DataFrame()
df['datetime'] = weather['datetime']
df['east_side'] = bike_flow['Fremont Bridge East Sidewalk']
df['west_side'] = bike_flow['Fremont Bridge West Sidewalk']
df['temperature'] = weather['temperature']
df['humidity'] = weather['humidity']
df['pressure'] = weather['pressure']
df['description'] = weather['description']
df['wind_direction'] = weather['wind_direction']
df['wind_speed'] = weather['wind_speed']
df.head()

,datetime,east_side,west_side,temperature,humidity,pressure,description,wind_direction,wind_speed
0,2012-11-30 00:00:00,2.0,5.0,7.42,87.0,1003.0,light rain,0.0,0.0
1,2012-11-30 01:00:00,1.0,0.0,7.78,87.0,1003.0,light rain,0.0,1.0
2,2012-11-30 02:00:00,1.0,0.0,8.13,87.0,1003.0,light rain,160.0,2.0
3,2012-11-30 03:00:00,1.0,0.0,8.24,64.0,1025.0,sky is clear,270.0,10.0
4,2012-11-30 04:00:00,2.0,6.0,8.66,68.0,1019.0,overcast clouds,309.0,5.0


In [5]:
df.shape

(43825, 9)

In [6]:
df['datetime'] = df['datetime'].astype('datetime64[ns]')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43825 entries, 0 to 43824
Data columns (total 9 columns):
datetime          43825 non-null datetime64[ns]
east_side         43817 non-null float64
west_side         43817 non-null float64
temperature       43823 non-null float64
humidity          43563 non-null float64
pressure          43814 non-null float64
description       43825 non-null object
wind_direction    43825 non-null float64
wind_speed        43825 non-null float64
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 3.0+ MB


### Adding new columns for futher analysis

In [7]:
from datetime import date
import holidays
#add new colunms Work Status and Day_Of_Week and Hour
weekStatus = np.empty(43825)
dayOfWeek = np.empty(43825)
hour = np.empty(43825)
for i in list(range(0,43825)):
    dayOfWeek[i] = df.iloc[i]['datetime'].isoweekday()
    hour[i] = df.iloc[i]['datetime'].hour
    if dayOfWeek[i] < 6:
     weekStatus[i] = 1
    else:
     weekStatus[i] = 0
for i in list(range(0,43825)):
    if df.iloc[i]['datetime'].date() in holidays.US():
        weekStatus[i] = 0

df = df.assign(Work_Status = weekStatus)

df = df.assign(Day_of_Week = dayOfWeek)
df['Day_of_Week'] = df['Day_of_Week'].astype('category')
df['Day_of_Week'].cat.categories = ["Mon", "Tue", "Wed", "Thu", "Fri","Sat","Sun"])
df = df.assign(Hour = hour)
df['Hour'] = df['Hour'].astype('category')
df.head()

,datetime,east_side,west_side,temperature,humidity,pressure,description,wind_direction,wind_speed,Work_Status,Day_of_Week,Hour
0,2012-11-30 00:00:00,2.0,5.0,7.42,87.0,1003.0,light rain,0.0,0.0,1.0,Fri,0.0
1,2012-11-30 01:00:00,1.0,0.0,7.78,87.0,1003.0,light rain,0.0,1.0,1.0,Fri,1.0
2,2012-11-30 02:00:00,1.0,0.0,8.13,87.0,1003.0,light rain,160.0,2.0,1.0,Fri,2.0
3,2012-11-30 03:00:00,1.0,0.0,8.24,64.0,1025.0,sky is clear,270.0,10.0,1.0,Fri,3.0
4,2012-11-30 04:00:00,2.0,6.0,8.66,68.0,1019.0,overcast clouds,309.0,5.0,1.0,Fri,4.0


In [8]:
df.isnull().sum()

datetime            0
east_side           8
west_side           8
temperature         2
humidity          262
pressure           11
description         0
wind_direction      0
wind_speed          0
Work_Status         0
Day_of_Week         0
Hour                0
dtype: int64

In [9]:
df["Date"] = [d.date() for d in df["datetime"]]
df.head()

,datetime,east_side,west_side,temperature,humidity,pressure,description,wind_direction,wind_speed,Work_Status,Day_of_Week,Hour,Date
0,2012-11-30 00:00:00,2.0,5.0,7.42,87.0,1003.0,light rain,0.0,0.0,1.0,Fri,0.0,2012-11-30
1,2012-11-30 01:00:00,1.0,0.0,7.78,87.0,1003.0,light rain,0.0,1.0,1.0,Fri,1.0,2012-11-30
2,2012-11-30 02:00:00,1.0,0.0,8.13,87.0,1003.0,light rain,160.0,2.0,1.0,Fri,2.0,2012-11-30
3,2012-11-30 03:00:00,1.0,0.0,8.24,64.0,1025.0,sky is clear,270.0,10.0,1.0,Fri,3.0,2012-11-30
4,2012-11-30 04:00:00,2.0,6.0,8.66,68.0,1019.0,overcast clouds,309.0,5.0,1.0,Fri,4.0,2012-11-30


### Filling NaN value with mean value of the Date

In [10]:
#filling missing value by the mean of that day
df1 = df.groupby([df.Date]).transform(lambda x: x.fillna(x.mean()))

In [11]:
df1.isnull().sum()

east_side         0
west_side         0
temperature       0
humidity          0
pressure          0
wind_direction    0
wind_speed        0
Work_Status       0
dtype: int64

In [12]:
result = pd.DataFrame()
result['date'] = df['Date']
result['hour'] = df['Hour']
result['east_side'] = df1['east_side']
result['west_side'] = df1['west_side']
result['temperature'] = df1['temperature']
result['humidity'] = df1['humidity']
result['pressure'] = df1['pressure']
result['description'] = df['description']
result['wind_direction'] = df1['wind_direction']
result['wind_speed'] = df1['wind_speed']
result['work_status'] = df['Work_Status']
result['day_of_week'] = df['Day_of_Week']
result.head()

,date,hour,east_side,west_side,temperature,humidity,pressure,description,wind_direction,wind_speed,work_status,day_of_week
0,2012-11-30,0.0,2.0,5.0,7.42,87.0,1003.0,light rain,0.0,0.0,1.0,Fri
1,2012-11-30,1.0,1.0,0.0,7.78,87.0,1003.0,light rain,0.0,1.0,1.0,Fri
2,2012-11-30,2.0,1.0,0.0,8.13,87.0,1003.0,light rain,160.0,2.0,1.0,Fri
3,2012-11-30,3.0,1.0,0.0,8.24,64.0,1025.0,sky is clear,270.0,10.0,1.0,Fri
4,2012-11-30,4.0,2.0,6.0,8.66,68.0,1019.0,overcast clouds,309.0,5.0,1.0,Fri


In [13]:
result.isnull().sum()

date              0
hour              0
east_side         0
west_side         0
temperature       0
humidity          0
pressure          0
description       0
wind_direction    0
wind_speed        0
work_status       0
day_of_week       0
dtype: int64

### writing to a new csv file

In [14]:
#file = 'complete.csv'
#result.to_csv(file,index=False, sep=',', encoding='utf-8')

In [15]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43825 entries, 0 to 43824
Data columns (total 12 columns):
date              43825 non-null object
hour              43825 non-null category
east_side         43825 non-null float64
west_side         43825 non-null float64
temperature       43825 non-null float64
humidity          43825 non-null float64
pressure          43825 non-null float64
description       43825 non-null object
wind_direction    43825 non-null float64
wind_speed        43825 non-null float64
work_status       43825 non-null float64
day_of_week       43825 non-null category
dtypes: category(2), float64(8), object(2)
memory usage: 3.4+ MB


In [16]:
season = []
for i in list(range(0,43825)):
    if result.iloc[i]['date'].month in [1,2,3,4,12]:
        season.append('Winter')
    if result.iloc[i]['date'].month in [5,6]:
        season.append('Spring')
    if result.iloc[i]['date'].month in [7,8,9]:
        season.append('Summer')
    if result.iloc[i]['date'].month in [10,11]:
        season.append('Fall')
result = result.assign(season = season)
result['season'] = result['season'].astype('category')

In [17]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43825 entries, 0 to 43824
Data columns (total 13 columns):
date              43825 non-null object
hour              43825 non-null category
east_side         43825 non-null float64
west_side         43825 non-null float64
temperature       43825 non-null float64
humidity          43825 non-null float64
pressure          43825 non-null float64
description       43825 non-null object
wind_direction    43825 non-null float64
wind_speed        43825 non-null float64
work_status       43825 non-null float64
day_of_week       43825 non-null category
season            43825 non-null category
dtypes: category(3), float64(8), object(2)
memory usage: 3.5+ MB


In [18]:
result = result.drop(['date', 'description'], axis=1)

In [19]:
result.head()

,hour,east_side,west_side,temperature,humidity,pressure,wind_direction,wind_speed,work_status,day_of_week,season
0,0.0,2.0,5.0,7.42,87.0,1003.0,0.0,0.0,1.0,Fri,Fall
1,1.0,1.0,0.0,7.78,87.0,1003.0,0.0,1.0,1.0,Fri,Fall
2,2.0,1.0,0.0,8.13,87.0,1003.0,160.0,2.0,1.0,Fri,Fall
3,3.0,1.0,0.0,8.24,64.0,1025.0,270.0,10.0,1.0,Fri,Fall
4,4.0,2.0,6.0,8.66,68.0,1019.0,309.0,5.0,1.0,Fri,Fall


In [20]:
#file = 'complete2.csv'
#result.to_csv(file,index=False, sep=',', encoding='utf-8')

In [24]:
df_all = result.copy()
df_all = df_all.drop(['east_side', 'west_side'], axis=1)
df_all.head()

,hour,temperature,humidity,pressure,wind_direction,wind_speed,work_status,day_of_week,season
0,0.0,7.42,87.0,1003.0,0.0,0.0,1.0,Fri,Fall
1,1.0,7.78,87.0,1003.0,0.0,1.0,1.0,Fri,Fall
2,2.0,8.13,87.0,1003.0,160.0,2.0,1.0,Fri,Fall
3,3.0,8.24,64.0,1025.0,270.0,10.0,1.0,Fri,Fall
4,4.0,8.66,68.0,1019.0,309.0,5.0,1.0,Fri,Fall


In [25]:
sample = df_all.iloc[0:8760]
sample.shape

(8760, 9)

In [26]:
file = 'sample.csv'
sample.to_csv(file,index=False, sep=',', encoding='utf-8')